In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

## Day 46 Lecture 1 Assignment

In this assignment, we will calculate a distance matrix for geographical Starbucks data and use it to identify locations that are close together and far apart. We will perform clustering on this dataset later on.

We will be using the "haversine" package to compute geographical distance. It can be pip installed.

In [2]:
!pip install haversine

<IPython.core.display.Javascript object>

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from haversine import haversine

<IPython.core.display.Javascript object>

Below is a convenience function for calculating geographical distance matrices using lat-long data.

In [4]:
def geo_sim_matrix(df, col_name = 'Coordinates'):
    """
    A function that computes a geographical distance matrix (in miles).
    Each row in the dataframe should correspond to one location.
    In addition, the dataframe must have a column containing the lat-long of each location as a tuple (i.e. (lat, long)).
    
    Parameters:
        df (pandas dataframe): an nxm dataframe containing the locations to compute similarities between.
        col (string): the name of the column containing the lat-long tuples.
        
    Returns:
        distance (pandas dataframe): an nxn distance matrix between the geographical coordinates of each location.
    """
    
    df = df.copy()
    df.reset_index(inplace=True)
    haver_vec = np.vectorize(haversine, otypes=[np.float32])
    distance = df.groupby('index').apply(lambda x: pd.Series(haver_vec(df[col_name], x[col_name])))
    distance = distance / 1.609344  # converts to miles
    distance.columns = distance.index
    
    return distance


<IPython.core.display.Javascript object>

This dataset contains the latitude and longitude (as well as several other details we will not be using) of every Starbucks in the world as of February 2017. Each row consists of the following features, which are generally self-explanatory:

- Brand
- Store Number
- Store Name
- Ownership Type
- Street Address
- City
- State/Province
- Country
- Postcode
- Phone Number
- Timezone
- Longitude
- Latitude

Load in the dataset.

In [7]:
# answer goes here
df = pd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/starbucks_locations.csv"
)
df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


<IPython.core.display.Javascript object>

Begin by narrowing down the dataset to a specific geographic area of interest. Since we will need to manually compute a distance matrix, which will be on the order of $n^{2}$ in terms of size, we would recommend choosing an area with 3000 or less locations. In this example, we will use Hawaii, which has about 100 locations; for reference, California has about 2800 locations. Feel free to choose a different region that is of more interest to you, if desired.

Subset the dataframe to only include records for Starbucks locations in Hawaii.

In [16]:
df = df[df["State/Province"] == "IL"]
df.head()
df["State/Province"].value_counts()

IL    575
Name: State/Province, dtype: int64

<IPython.core.display.Javascript object>

The haversine package takes tuples with 2 numeric elements and interprets them as lat-long to calculate distance, so add a new column called "Coordinates" that converts the lat and long in each row into a tuple. In other words, the last two columns of the dataframe should initially look like this:

**Latitude, Longitude**  
39.14, -121.64  
34.13, -116.40  
...

After adding the new column, the last three columns should look like this:

**Latitude, Longitude, Coordinates**  
39.14, -121.64,   (39.14, -121.64)  
34.13, -116.40,   (34.13, -116.40)  
...

In [22]:
# answer goes here
df["Coordinates"] = list(zip(df["Latitude"], df["Longitude"]))
df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates
17457,Starbucks,2726-85202,Lake & Rohlwing,Company Owned,"1590 W. Lake St., Suite 101",Addison,IL,US,601011824,630773-4743,GMT-06:00 America/Chicago,-88.03,41.95,"(41.95, -88.03)"
17458,Starbucks,76107-79323,Super Target Algonquin ST-1801,Licensed,"750 Randall Rd, Ryan Place",Algonquin,IL,US,60102,847-458-5340,GMT-06:00 America/Chicago,-88.34,42.17,"(42.17, -88.34)"
17459,Starbucks,2626-63429,East Algonquin & Ryan,Company Owned,"1532 E. Algonquin Road, Floor 1",Algonquin,IL,US,601024519,847-458-6740,GMT-06:00 America/Chicago,-88.27,42.16,"(42.16, -88.27)"
17460,Starbucks,18024-182807,Randall & Harnish,Company Owned,"790 S. Randall Road, Suite 1",Algonquin,IL,US,60102,8474580669,GMT-06:00 America/Chicago,-88.34,42.16,"(42.16, -88.34)"
17461,Starbucks,21358-211207,Arlington Heights & Golf Road,Company Owned,1808 South Arlington Heights Road,Arlington Heights,IL,US,60005,8476402779,GMT-06:00 America/Chicago,-87.98,42.05,"(42.05, -87.98)"


<IPython.core.display.Javascript object>

Calculate the distance matrix using the starter code/function geo_sim_matrix() provided earlier in the notebook. It assumes the column containing the coordinates for each location is called "Coordinates". Examine the docstring for more details.

Note: the latitude and longitudes provided only go out to two decimal places, which limits the resolution of the distance calculations to about 0.5 miles. Distances that are very small may not be accurately represented here (e.g. several instances of "0 distance" for distinct Starbucks locations in very close proximity).

In [26]:
# answer goes here
sim_matrix = geo_sim_matrix(df, col_name="Coordinates")
sim_matrix.head()

index,17457,17458,17459,17460,17461,17462,17463,17464,17465,17466,...,18022,18023,18024,18025,18026,18027,18028,18029,18030,18031
index,,,,,,,,,,,,,,,,,,,,,
17457,0.000000,21.998655,19.029597,21.527960,7.370898,9.341678,9.686688,11.348866,21.130444,18.405280,...,16.516217,16.516217,19.506333,18.539901,13.856968,12.606001,12.479327,32.891659,29.349573,29.109865
17458,21.998655,0.000000,3.650891,0.690934,20.229834,19.468171,17.303818,18.904129,26.335757,30.067912,...,30.365815,30.365815,33.256965,31.018190,33.609257,30.922485,31.396745,11.190566,34.509039,35.610755
17459,19.029597,3.650891,0.000000,3.585198,16.696038,15.856924,13.693897,15.255140,26.181536,29.037429,...,26.715266,26.715266,29.607542,27.374610,31.320382,28.838915,29.220768,13.863712,34.722152,35.635412
17460,21.527960,0.690934,3.585198,0.000000,19.957956,19.260005,17.096893,18.766155,25.647000,29.385448,...,30.234197,30.234197,33.151695,30.935839,33.015211,30.307037,30.790857,11.793137,33.831552,34.927262
17461,7.370898,20.229834,16.696038,19.957956,0.000000,2.072803,3.441989,4.145605,27.305780,25.526491,...,11.473452,11.473452,14.624680,12.987062,20.785362,19.891942,19.678132,30.030112,35.905930,35.868051


<IPython.core.display.Javascript object>

For each Starbucks, identify its nearest neighboring location in Hawaii (and presumably in the world). Save the output to a dataframe with three columns: Location, Nearest Neighbor, and Distance (Miles).

In [27]:
# answer goes here
np.fill_diagonal(sim_matrix.values, np.nan)
df["Nearest Neighbor"] = sim_matrix.idxmin()
df["Distance (Miles)"] = sim_matrix.min()
df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,Nearest Neighbor,Distance (Miles)
17457,Starbucks,2726-85202,Lake & Rohlwing,Company Owned,"1590 W. Lake St., Suite 101",Addison,IL,US,601011824,630773-4743,GMT-06:00 America/Chicago,-88.03,41.95,"(41.95, -88.03)",17797,1.722053
17458,Starbucks,76107-79323,Super Target Algonquin ST-1801,Licensed,"750 Randall Rd, Ryan Place",Algonquin,IL,US,60102,847-458-5340,GMT-06:00 America/Chicago,-88.34,42.17,"(42.17, -88.34)",17460,0.690934
17459,Starbucks,2626-63429,East Algonquin & Ryan,Company Owned,"1532 E. Algonquin Road, Floor 1",Algonquin,IL,US,601024519,847-458-6740,GMT-06:00 America/Chicago,-88.27,42.16,"(42.16, -88.27)",17807,3.368986
17460,Starbucks,18024-182807,Randall & Harnish,Company Owned,"790 S. Randall Road, Suite 1",Algonquin,IL,US,60102,8474580669,GMT-06:00 America/Chicago,-88.34,42.16,"(42.16, -88.34)",17458,0.690934
17461,Starbucks,21358-211207,Arlington Heights & Golf Road,Company Owned,1808 South Arlington Heights Road,Arlington Heights,IL,US,60005,8476402779,GMT-06:00 America/Chicago,-87.98,42.05,"(42.05, -87.98)",17934,1.026121


<IPython.core.display.Javascript object>

If the nearest neighbor of a Starbucks location is far away, we could consider that Starbucks to be "on an island". Which five Starbucks in Hawaii are the most "on an island"?

In [31]:
# answer goes here
df.sort_values('Distance (Miles)', ascending = False).tail()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Coordinates,Nearest Neighbor,Distance (Miles)
17662,Starbucks,19549-197684,Erie & Saint Clair - Hyatt Hotel,Company Owned,633 North Saint Clair Street,Chicago,IL,US,60611,3127872038,GMT-06:00 America/Chicago,-87.62,41.89,"(41.89, -87.62)",17523,0.0
17661,Starbucks,2527-32799,Madison & Morgan,Company Owned,1001A West Madison Avenue,Chicago,IL,US,606072007,312-421-7073,GMT-06:00 America/Chicago,-87.65,41.88,"(41.88, -87.65)",17516,0.0
17660,Starbucks,11719-103965,Halstead & Monroe - Greek Town,Company Owned,"116 S. Halsted St. #E, Corner of Madison & Morgan",Chicago,IL,US,606613508,312-226-3658,GMT-06:00 America/Chicago,-87.65,41.88,"(41.88, -87.65)",17516,0.0
17878,Starbucks,75676-90374,Marriott Oakbrook,Licensed,1401 W. 22nd Street,Oak Brook,IL,US,605232009,630-573-8555,GMT-06:00 America/Chicago,-87.95,41.85,"(41.85, -87.95)",17880,0.0
17564,Starbucks,72403-3869,ORD OHare Concourse E,Licensed,"Circular Building #6, Guard Post 8",Chicago,IL,US,60666,773-686-6180,GMT-06:00 America/Chicago,-87.91,41.98,"(41.98, -87.91)",17537,0.0


<IPython.core.display.Javascript object>

In [32]:
df["City"].value_counts()

Chicago          180
Schaumburg        12
Naperville        12
Peoria             9
Evanston           8
                ... 
Bartlett           1
Volo               1
Bensenville        1
Batavia            1
South Holland      1
Name: City, Length: 174, dtype: int64

<IPython.core.display.Javascript object>